In [1]:
import pandas as pd
import numpy as np

In [2]:
dat = pd.read_excel("./data/test_reshape.xlsx")

In [3]:
dat

,ID,Name,gp1_a,gp1_b,gp1_c,gp2_x,gp2_y
0,1,Alex,1.0,NaN,NaN,NaN,NaN
1,2,Bob,NaN,NaN,1.0,NaN,NaN
2,3,Catherine,NaN,1.0,1.0,NaN,NaN
3,4,Dave,1.0,NaN,NaN,1.0,NaN
4,5,Edward,NaN,NaN,NaN,NaN,1.0


In [4]:
tidy = pd.melt(dat, id_vars=['ID', 'Name'], var_name="class", value_name="value").dropna(subset=['value'])
tidy.sort_values(by=['ID'])

,ID,Name,class,value
0,1,Alex,gp1_a,1.0
11,2,Bob,gp1_c,1.0
7,3,Catherine,gp1_b,1.0
12,3,Catherine,gp1_c,1.0
3,4,Dave,gp1_a,1.0
18,4,Dave,gp2_x,1.0
24,5,Edward,gp2_y,1.0


In [5]:
tidy['group'] = tidy['class'].apply(lambda s: s[:3])
tidy['subclass'] = tidy['class'].map(lambda s: s[-1:])
tidy.sort_values(by='ID')

,ID,Name,class,value,group,subclass
0,1,Alex,gp1_a,1.0,gp1,a
11,2,Bob,gp1_c,1.0,gp1,c
7,3,Catherine,gp1_b,1.0,gp1,b
12,3,Catherine,gp1_c,1.0,gp1,c
3,4,Dave,gp1_a,1.0,gp1,a
18,4,Dave,gp2_x,1.0,gp2,x
24,5,Edward,gp2_y,1.0,gp2,y


In [6]:
tidy = tidy.drop(columns=['class', 'value'])

In [7]:
tidy

,ID,Name,group,subclass
0,1,Alex,gp1,a
3,4,Dave,gp1,a
7,3,Catherine,gp1,b
11,2,Bob,gp1,c
12,3,Catherine,gp1,c
18,4,Dave,gp2,x
24,5,Edward,gp2,y


In [10]:
tidy1 = tidy.groupby(['ID','Name']).apply(pd.DataFrame.pivot,
                              # index=None,
                              columns='group',
                              values='subclass')
tidy1

gp1   gp2
ID Name                    
1  Alex      0      a   NaN
2  Bob       11     c   NaN
3  Catherine 7      b   NaN
             12     c   NaN
4  Dave      3      a  None
             18  None     x
5  Edward    24   NaN     y

In [11]:
tidy1.reset_index(inplace=True)
tidy2 = tidy1.drop(columns='level_2')
tidy2

,ID,Name,gp1,gp2
0,1,Alex,a,NaN
1,2,Bob,c,NaN
2,3,Catherine,b,NaN
3,3,Catherine,c,NaN
4,4,Dave,a,None
5,4,Dave,None,x
6,5,Edward,NaN,y


In [12]:
tidy2.dtypes

ID       int64
Name    object
gp1     object
gp2     object
dtype: object

In [26]:
a = tidy2.gp1[6]
b = tidy2.gp1[5]
c = 'xyz'

# print([a,b,c])
','.join(str(x) for x in [a,b,c])

'nan,None,xyz'

In [25]:
# tidy3 = tidy2.groupby(['ID', 'Name'])['gp1'].apply(','.join).reset_index()
tidy3 = tidy2.groupby(['ID', 'Name'])['gp1'].apply(lambda x: "{%s}" % ', '.join(str(x)))
tidy3

ID  Name     
1   Alex         {0,  ,  ,  ,  , a, \n, N, a, m, e, :,  , (, 1,...
2   Bob          {1,  ,  ,  ,  , c, \n, N, a, m, e, :,  , (, 2,...
3   Catherine    {2,  ,  ,  ,  , b, \n, 3,  ,  ,  ,  , c, \n, N...
4   Dave         {4,  ,  ,  ,  ,  ,  ,  , a, \n, 5,  ,  ,  ,  ,...
5   Edward       {6,  ,  ,  ,  , N, a, N, \n, N, a, m, e, :,  ,...
Name: gp1, dtype: object

**Reference:**
- https://www.ibm.com/developerworks/community/blogs/jfp/entry/Tidy_Data_In_Python?lang=en